This notebook converts tiled geotiffs of the various datasets in Zeno project to 
zarr format for efficient read for large scale zonal stats analysis.

See `compare_zarr_to_cog.ipynb` for a performance benchmark between tiled geotiff, cog and zarr formats

In [7]:
import coiled

import fsspec
import numpy as np
import rioxarray
import xarray as xr
import fsspec
import pandas as pd
import logging
from flox.xarray import xarray_reduce
import numpy as np

import dask
import zarr
import gcsfs
import boto3
import warnings

In [8]:
fs = fsspec.filesystem("s3", requester_pays=True)

In [9]:
logging.getLogger("distributed.client").setLevel(logging.ERROR)

In [10]:
cluster = coiled.Cluster(
    name="tcl_dask",
    region="us-east-1",
    n_workers=20,
    tags={"project": "tcl_dask"},
    scheduler_vm_types="r7g.xlarge",
    worker_vm_types="r7g.2xlarge",
    compute_purchase_option="spot_with_fallback"
)

client = cluster.get_client()

[2025-07-24 13:47:44,799][INFO    ][coiled] Fetching latest package priorities...
[2025-07-24 13:47:44,802][INFO    ][coiled.package_sync] Resolving your local zeno Python environment...
[2025-07-24 13:47:45,520][INFO    ][coiled.package_sync] Scanning 405 conda packages...
[2025-07-24 13:47:45,536][INFO    ][coiled.package_sync] Scanning 251 python packages...
[2025-07-24 13:47:46,866][INFO    ][coiled.software_utils] No username or password found for https://conda.anaconda.org/conda-forge
[2025-07-24 13:47:48,179][INFO    ][coiled] Running pip check...
[2025-07-24 13:47:49,514][INFO    ][coiled] Validating environment...
[2025-07-24 13:48:17,646][WARNING ][coiled.package_sync] Package - pydantic_core, pydantic-core~=2.33.2 has no install candidate for Python 3.12 linux-aarch64 on conda-forge
[2025-07-24 13:48:17,648][INFO    ][coiled] Requesting package sync build...
[2025-07-24 13:48:18,541][INFO    ][coiled] Creating Cluster (name: tcl_dask, https://cloud.coiled.io/clusters/1051629

In [11]:
cluster.adapt(minimum=10, maximum=50)

2025-07-24 13:49:56,048 - distributed.deploy.adaptive - INFO - Adaptive scaling started: minimum=10 maximum=50


In [6]:
dist_alerts_tiles = pd.read_json(
    "s3://gfw-data-lake/umd_glad_dist_alerts/v20250510/raster/epsg-4326/10/40000/default/gdal-geotiff/tiles.geojson"
)
adm0_tiles = pd.read_json(
    's3://gfw-data-lake/gadm_administrative_boundaries/v4.1.85/raster/epsg-4326/10/40000/adm0/gdal-geotiff/tiles.geojson'
)

adm1_tiles = pd.read_json(
    's3://gfw-data-lake/gadm_administrative_boundaries/v4.1.85/raster/epsg-4326/10/40000/adm1/gdal-geotiff/tiles.geojson'
)

adm2_tiles = pd.read_json(
    's3://gfw-data-lake/gadm_administrative_boundaries/v4.1.85/raster/epsg-4326/10/40000/adm2/gdal-geotiff/tiles.geojson'
)

pixel_area_tiles = pd.read_json(
    's3://gfw-data-lake/umd_area_2013/v1.10/raster/epsg-4326/10/40000/area_m/gdal-geotiff/tiles.geojson'
)

def get_uri(feature):
    raw = feature['properties']['name'].split('/')[2:]
    uri = '/'.join(['s3:/'] + raw)
    return uri

dist_alerts_tile_uris = dist_alerts_tiles.features.apply(get_uri)
adm0_tile_uris = adm0_tiles.features.apply(get_uri)
adm1_tile_uris = adm1_tiles.features.apply(get_uri)
adm2_tile_uris = adm2_tiles.features.apply(get_uri)
pixel_area_uris = pixel_area_tiles.features.apply(get_uri)


In [ ]:
dist_alerts = tcl_year = xr.open_mfdataset(
    dist_alerts_tile_uris,
    parallel=True,
    chunks={'x': 10000, 'y':10000}
).astype(np.int16)

In [ ]:
dist_alerts.band_data

In [ ]:
dist_zarr_name = "s3://gfw-data-lake/umd_glad_dist_alerts/v20250510/raster/epsg-4326/zarr/dist_alerts_full.zarr"

In [ ]:
dist_alerts.band_data.to_zarr(dist_zarr_name, mode='w')

#### Save alert date and confidence as separate variables as well

In [ ]:
alert_date = dist_alerts.band_data % 10000
alert_conf = (dist_alerts.band_data // 10000).astype(np.uint8)
alert_conf.name = "confidence"
alert_date.name = "alert_date"
date_conf = xr.merge((alert_conf, alert_date))
date_conf.to_zarr("s3://gfw-data-lake/umd_glad_dist_alerts/v20250510/raster/epsg-4326/zarr/date_conf.zarr", mode="w")

In [ ]:

adm0 = xr.open_mfdataset(
    adm0_tile_uris,
    parallel=True,
    chunks={'x': 10000, 'y':10000}
).astype(np.uint16)

In [ ]:
adm0.band_data

In [ ]:
adm1 = xr.open_mfdataset(
    adm1_tile_uris,
    parallel=True,
    chunks={'x': 10000, 'y':10000}
).astype(np.uint8)

In [ ]:
adm1.band_data

In [ ]:
adm2 = xr.open_mfdataset(
    adm2_tile_uris,
    parallel=True,
    chunks={'x': 10000, 'y':10000}
).astype(np.uint16)

In [ ]:
adm2.band_data

In [7]:
pixel_area = xr.open_mfdataset(
    pixel_area_uris,
    parallel=True,
    chunks={'x': 10000, 'y':10000}
)
pixel_area.band_data

<xarray.DataArray 'band_data' (band: 1, y: 560000, x: 1440000)> Size: 3TB
dask.array<concatenate, shape=(1, 560000, 1440000), dtype=float32, chunksize=(1, 10000, 10000), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 12MB -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 4MB 80.0 80.0 80.0 80.0 ... -60.0 -60.0 -60.0 -60.0
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:             Area
    STATISTICS_MAXIMUM:        266.26049804688
    STATISTICS_MEAN:           201.35177368655
    STATISTICS_MINIMUM:        135.3455657959
    STATISTICS_STDDEV:         37.813954802425
    STATISTICS_VALID_PERCENT:  100

In [ ]:
_, adm1_aligned = xr.align(dist_alerts, adm1, join='left')

In [ ]:
adm1_aligned.band_data.to_zarr(
    's3://gfw-data-lake/gadm_administrative_boundaries/v4.1.85/raster/epsg-4326/zarr/adm1_clipped_to_dist.zarr'
)

In [ ]:
_, adm2_aligned = xr.align(dist_alerts, adm2, join='left')

In [ ]:
adm2_aligned.band_data.to_zarr(
    's3://gfw-data-lake/gadm_administrative_boundaries/v4.1.85/raster/epsg-4326/zarr/adm2_clipped_to_dist.zarr'
)

In [ ]:
_, adm0_aligned = xr.align(dist_alerts, adm0, join='left')

In [ ]:
adm0_aligned.band_data.to_zarr(
    's3://gfw-data-lake/gadm_administrative_boundaries/v4.1.85/raster/epsg-4326/zarr/adm0_clipped_to_dist.zarr', mode='w'
)

In [ ]:
_, pixel_area_aligned = xr.align(dist_alerts, pixel_area, join='left')

In [ ]:
pixel_area_aligned.band_data

In [ ]:
pixel_area_aligned.band_data.to_zarr(
    's3://gfw-data-lake/umd_area_2013/v1.10/raster/epsg-4326/zarr/pixel_area_clipped_to_dist.zarr', mode='w'
)

In [ ]:
def set_env():
    import os
    os.environ['GS_NO_SIGN_REQUEST'] = 'YES'

client.run(set_env)

In [ ]:
gfs = gcsfs.GCSFileSystem(token=None)

bucket_path = 'lcl_public/SBTN_NaturalLands/v1_1/classification/'
file_list = gfs.glob(f'{bucket_path}*.tif')

natural_lands_urls_all_classes_urls = [f'gs://{f}' for f in file_list]

In [ ]:
sbtn_natural_lands_all_classes = xr.open_mfdataset(
    natural_lands_urls_all_classes_urls,
    parallel=True,
    chunks={'x': 10000, 'y':10000}
).astype(np.uint8)

In [ ]:
sbtn_natural_lands_all_classes.band_data

In [ ]:
_, sbtn_natural_lands_all_classes_clipped = xr.align(dist_alerts, sbtn_natural_lands_all_classes, join='left')

In [ ]:
sbtn_natural_lands_all_classes_clipped.to_zarr(
    "s3://gfw-data-lake/sbtn_natural_lands/zarr/sbtn_natural_lands_all_classes_clipped_to_dist.zarr", mode="w"
)

### Natural Grasslands Zarr

In [3]:
s3 = boto3.client("s3")
response = s3.list_objects_v2(Bucket="gfw-data-lake", Prefix="gfw_grasslands/v1.1/geotiff")
if "Contents" in response:
    grasslands_uris = [f"s3://gfw-data-lake/{content['Key']}" for content in response["Contents"] if content["Key"].endswith(".tif")]

In [23]:
annual_grasslands = []
for uri in grasslands_uris:
    year = int(uri[-8:-4])
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        ds = xr.open_dataset(uri, engine="rasterio", chunks={"x": 10000, "y": 10000}).astype(np.uint8)
    ds = ds.expand_dims(year=[year])
    annual_grasslands.append(ds)

In [24]:
# concatenate
grasslands = xr.concat(annual_grasslands, dim="year")

In [25]:
grasslands.band_data

<xarray.DataArray 'band_data' (year: 25, band: 1, y: 528004, x: 1436004)> Size: 19TB
dask.array<concatenate, shape=(25, 1, 528004, 1436004), dtype=uint8, chunksize=(1, 1, 10000, 10000), chunktype=numpy.ndarray>
Coordinates:
  * year         (year) int64 200B 2000 2001 2002 2003 ... 2021 2022 2023 2024
  * band         (band) int64 8B 1
  * x            (x) float64 11MB -179.0 -179.0 -179.0 ... 180.0 180.0 180.0
  * y            (y) float64 4MB 76.0 76.0 76.0 76.0 ... -56.0 -56.0 -56.0 -56.0
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area

In [26]:
grasslands.to_zarr("s3://gfw-data-lake/gfw_grasslands/v1.1/zarr/grasslands_2000_2024.zarr", mode="w")

/Users/maanas.nukala/miniforge3/envs/zeno/lib/python3.12/site-packages/zarr/api/asynchronous.py:213: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(
/Users/maanas.nukala/miniforge3/envs/zeno/lib/python3.12/site-packages/distributed/client.py:3363: UserWarning: Sending large graph of size 15.19 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


[2025-07-24 13:52:31,835][INFO    ][coiled] Adaptive is removing 10 workers.
[2025-07-24 13:52:47,277][INFO    ][coiled] Adaptive scaling up to 50 workers.
[2025-07-24 14:35:56,201][INFO    ][coiled] Adaptive is removing 32 workers.
